In [ ]:
#cose da eseguire
#in un terminale (1) -> docker-compose up
#in un altro terminale (2) -> docker exec -it ollama bash
#nel terminale 2 -> ollama pull phi3


In [ ]:
!pip install elasticsearch tqdm openai

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [ ]:
import json

#create the docs_raw variable from the .json file
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
        

In [ ]:
documents[0]

In [ ]:
def build_prompt(query,search_results):
    prompt_template="""
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't cointain the answer, output NONE.

    QUESTION: {question}

    CONTEXT:
    {context}
    """

    
    context = ""

    for doc in search_results:
        context= context + f"section: {doc['section']}\n\nquestion: {doc['question']}\n\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query,context=context)
    return prompt

In [ ]:
def llm(prompt):

    response=client.chat.completions.create(
        model='qwen2:1.5b',
        messages=[{"role": "user", "content":prompt}]
    )
    return response.choices[0].message.content
    #import ollama
    #response = ollama.generate(model="gemma:2b", prompt=prompt)
    #return response

In [ ]:
print(response['response'])

In [ ]:
query = "the course has already started, can I still enroll?"


In [ ]:
def rag(query):
    results = search_elastic(query)
    prompt = build_prompt(query,results)
    response = llm(prompt)
    return response#['response']

In [111]:
answer = rag(query)
print(answer)

TypeError: list indices must be integers or slices, not str

#elastic search

In [106]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
index_name = "course-questions"
import tqdm
#dont run this cell several times, or (idk why) some doc duplicated. If so, try to delete the index ->
es_client.indices.delete(index=index_name)
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)



  0%|          | 0/948 [00:00<?, ?it/s]

In [103]:
#CREATING INDICES
#they are saved permanently, so you have to run it only 1 time, no need to rerun after power-up

index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "text"},
                "question": {"type": "text"},
                "course": {"type": "keyword"} 
            }
        }
    }
    

    
es_client.indices.create(index=index_name, body=index_settings)
#es_client.indices.delete(index=index_name)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [110]:
#"filter" chooose the document that match with the "course"
#"question^3" means that the field is 3 times more important than the other
#"size" is the number of document retrieve


def elastic_search(query, index_name = "course-questions"):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
#                "filter": {
#                    "term": {
#                        "course": "data-engineering-zoomcamp"
#                    }
#                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [112]:
results = elastic_search(query,index_name)

In [113]:
results



[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the

In [ ]:
es_client.ping()


In [ ]:
print(es_client.search(index=index_name, body={ 
        "size": 1000,  # Puoi modificare questo valore in base alle tue esigenze
        "query": {
            "bool": {
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
}
                      ))

In [ ]:

print(es_client.search(index=index_name, body={ 
        "size": 1000,  # Puoi modificare questo valore in base alle tue esigenze
        "query": {
            "match_all": {}
        }
    })['hits']['hits'])

In [101]:
mapping = es_client.indices.get_mapping(index=index_name)

mapping

ObjectApiResponse({'course-questions': {'mappings': {'properties': {'course': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'question': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'section': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'text': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}}})

In [102]:
es_client.indices.delete(index=index_name)

ObjectApiResponse({'acknowledged': True})